In [1]:
import yfinance as yf
import pandas as pd
import requests
from tqdm import tqdm

url = 'https://www.slickcharts.com/sp500'
headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
request = requests.get(url, headers = headers)
data = pd.read_html(request.text)[0]
stk_list = data.Symbol
stk_list = data.Symbol.apply(lambda x: x.replace('.', '-'))
# S&P 500 companies

In [2]:
# stk_list = list(stk_list)

In [3]:
tickers = yf.Tickers(' '.join(stk_list.tolist()))

In [9]:
getattr(tickers.tickers, "AMZN").info

{'zip': '98109-5210',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 798000,
 'longBusinessSummary': 'Amazon.com, Inc. engages in the retail sale of consumer products and subscriptions in North America and internationally. The company operates through three segments: North America, International, and Amazon Web Services (AWS). It sells merchandise and content purchased for resale from third-party sellers through physical and online stores. The company also manufactures and sells electronic devices, including Kindle, Fire tablets, Fire TVs, Rings, and Echo and other devices; provides Kindle Direct Publishing, an online service that allows independent authors and publishers to make their books available in the Kindle Store; and develops and produces media content. In addition, it offers programs that enable sellers to sell their products on its Websites, as well as its stores; and programs that allow authors, musicians, filmmakers, skill and app developers, and others to publish a

In [11]:
getattr(tickers.tickers, "AAPL").info

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 137000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides digital content stores and streaming services; AppleCare support services; and iCloud, a cloud service, which stores music, photos, contacts, calendars, mail, documents, and others. In addition, the company offers various service, such as Apple Arcade, a game subscription service; Apple Card, a co-branded credit card; Apple News+, a subscription news and magazine service; and Apple Pay, a cashless payment service, as well 

In [ ]:
info_ = getattr(tickers.tickers, "AMZN").info
print("info assigned")
sector = info_['sector']
print("sector assigned")
market_cap = info_['marketCap']
# print("mkt cap assigned")
# sectors.append(sector)
# print("sector appended")
# market_caps.append(market_cap)
# print("mkt cap appended")
# ticker_lst.append(t)
# print("ticker list appended")

In [6]:
sectors = []
market_caps = []
probs = []
ticker_lst = []
for t in tqdm(stk_list):
    print(t)
    try:
        info_ = getattr(tickers.tickers, t).info
        print("info assigned")
        sector = info_['sector']
        print("sector assigned")
        market_cap = info_['marketCap']
        print("mkt cap assigned")
        sectors.append(sector)
        print("sector appended")
        market_caps.append(market_cap)
        print("mkt cap appended")
        ticker_lst.append(t)
        print("ticker list appended")
    except (AttributeError, IndexError, ValueError, Exception) as e:
        probs.append((t, e))
        continue

  0%|          | 0/506 [00:00<?, ?it/s]

AAPL


  0%|          | 0/506 [00:26<?, ?it/s]


KeyboardInterrupt: 

In [7]:
probs

[('AAPL', NameError("name 'tickers' is not defined")),
 ('MSFT', NameError("name 'tickers' is not defined")),
 ('AMZN', NameError("name 'tickers' is not defined")),
 ('FB', NameError("name 'tickers' is not defined")),
 ('GOOGL', NameError("name 'tickers' is not defined")),
 ('GOOG', NameError("name 'tickers' is not defined")),
 ('BRK-B', NameError("name 'tickers' is not defined")),
 ('JNJ', NameError("name 'tickers' is not defined")),
 ('V', NameError("name 'tickers' is not defined")),
 ('PG', NameError("name 'tickers' is not defined")),
 ('NVDA', NameError("name 'tickers' is not defined")),
 ('HD', NameError("name 'tickers' is not defined")),
 ('JPM', NameError("name 'tickers' is not defined")),
 ('MA', NameError("name 'tickers' is not defined")),
 ('UNH', NameError("name 'tickers' is not defined")),
 ('VZ', NameError("name 'tickers' is not defined")),
 ('DIS', NameError("name 'tickers' is not defined")),
 ('PYPL', NameError("name 'tickers' is not defined")),
 ('ADBE', NameError("name

In [4]:
info_df = pd.DataFrame({'TICKER': ticker_lst, 'SECTOR': sectors, 'MARKET_CAPS': market_caps})
info_df.to_csv("info_df.csv", index=None)

In [5]:
dfs = []
unique_sectors = info_df['SECTOR'].unique().tolist()
for sector in unique_sectors:
    sector_df = info_df.loc[info_df['SECTOR'] == sector, :].copy()
    sector_df = sector_df.sort_values(by="MARKET_CAPS")[-1:-6:-1]
    dfs.append(sector_df)

top5_df = pd.concat(dfs)
top5_df.to_csv("top5_df.csv", index=None)    

In [6]:
top5_df.head(10)

TICKER             SECTOR    MARKET_CAPS
0    AAPL         Technology  2127040479232
1    MSFT         Technology  1612060819456
10   NVDA         Technology   312014110720
14   ADBE         Technology   226987065344
18   INTC         Technology   209587830784
2    AMZN  Consumer Cyclical  1645283377152
9      HD  Consumer Cyclical   304619552768
30    NKE  Consumer Cyclical   171197939712
27    MCD  Consumer Cyclical   157429874688
38    LOW  Consumer Cyclical   122260324352

In [7]:
top5_tickers = top5_df['TICKER'].tolist()

In [8]:
data = yf.download(tickers = top5_tickers, period = "3mo", interval = "1wk", # '1d'
                   group_by = 'ticker', auto_adjust = True, prepost = False, threads = True, proxy = None)

[*********************100%***********************]  55 of 55 completed

4 Failed downloads:
- NFLX: No data found, symbol may be delisted
- BLK: No data found, symbol may be delisted
- RTX: No data found, symbol may be delisted
- PG: No data found, symbol may be delisted


In [9]:
data.to_pickle("sector_top5_data.pkl")

In [10]:
data = pd.read_pickle("sector_top5_data.pkl")

In [11]:
data.head()

AEP                                               \
                 Open       High        Low      Close      Volume   
Date                                                                 
2020-05-18        NaN        NaN        NaN        NaN         NaN   
2020-05-21        NaN        NaN        NaN        NaN         NaN   
2020-05-25  78.848135  85.146066  78.659690  84.550987  13697200.0   
2020-06-01  84.382379  87.238761  82.696313  84.808853  10175400.0   
2020-06-08  84.729513  87.397459  79.958955  81.129280  12580400.0   

                   DLR                                                  ...  \
                  Open        High         Low       Close      Volume  ...   
Date                                                                    ...   
2020-05-18  130.737469  134.723856  128.734372  134.674271   3404800.0  ...   
2020-05-21         NaN         NaN         NaN         NaN         NaN  ...   
2020-05-25  138.254088  144.987315  133.662802  142.359467   8122200.0  ...   
2020-06-01  142.746196  143.727922  132.899244  140.098526  11562000.0  ...   
2020-06-08  140.237359  142.290056  131.699360  134.862686  11592100.0  ...   

                  COST                                                  \
                  Open        High         Low       Close      Volume   
Date                                                                     
2020-05-18         NaN         NaN         NaN         NaN         NaN   
2020-05-21         NaN         NaN         NaN         NaN         NaN   
2020-05-25  305.353211  310.332491  298.827239  307.807922  15503500.0   
2020-06-01  307.239141  314.673163  304.065974  311.370270  15918000.0   
2020-06-08  309.993220  310.621873  295.614145  298.058899  17096100.0   

                    HD                                                  
                  Open        High         Low       Close      Volume  
Date                                                                    
2020-05-18  236.209566  240.901551  234.390427  240.444290   7814500.0  
2020-05-21         NaN         NaN         NaN         NaN         NaN  
2020-05-25  246.677053  250.732838  239.350804  247.005096  19802100.0  
2020-06-01  247.929590  254.331367  244.410594  253.386993  23160600.0  
2020-06-08  252.490005  259.290009  237.050003  242.449997  22918800.0  

[5 rows x 283 columns]